### Load Data

In [315]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re as re
import math as math

filepath = "Data/train.csv"
titanic = pd.read_csv(filepath)

The code that follows is a detailed walkthrough of the wrangling module I created for the Titanic data.

### Separate Prefix and Numbers from Ticket

In [316]:
titanic["Ticket"] = titanic['Ticket'].str.replace('[^\w\s]','') #Replace punctuation with empty string

In [317]:
titanic["Ticket_Num"] = np.nan #Create new column for ticket numbers with NaN value
titanic["Ticket_Prefix"] = "" #Create new colum for ticket prefix letters with empty string

for index, row in titanic.iterrows(): #iterate over each row of titanic training data
    list_string = row["Ticket"].split() #split Ticket value into list of strings
    n = len(list_string) #store length of list
    if n == 1 and list_string[0].isdigit(): #store new ticket number value as integer of current value from list if there is only 1 string element and is numeric
        titanic.loc[index ,"Ticket_Num"] = int(list_string[0])
        
    elif n == 1: #store new ticket prefix value if there is only 1 element in list and it is not numeric
        titanic.loc[index ,"Ticket_Prefix"] = list_string[0]
        
    else: #store new ticket prefix and number values 
        titanic.loc[index ,"Ticket_Prefix"] = " ".join(list_string[0:n-1])
        titanic.loc[index ,"Ticket_Num"] = int(list_string[n-1])

In [318]:
titanic = titanic.drop('Ticket', 1) #drop Ticket column

In [319]:
titanic.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket_Num,Ticket_Prefix
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S,21171.0,A5
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,17599.0,PC
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S,3101282.0,STONO2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,113803.0,
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S,373450.0,


### Create Binary Column for Cabin

In [320]:
titanic["Cabin_Yes"] = 0

for index, row in titanic.iterrows():
    if type(row["Cabin"]) != float:
        titanic.loc[index,'Cabin_Yes'] = 1

In [321]:
titanic.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket_Num,Ticket_Prefix,Cabin_Yes
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S,21171.0,A5,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,17599.0,PC,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S,3101282.0,STONO2,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,113803.0,,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S,373450.0,,0


### Determine Unique Cabin Letters

In [322]:
titanic.Cabin.unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6', 'C23 C25 C27',
       'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33', 'F G73', 'E31',
       'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101', 'F E69', 'D47',
       'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4', 'A32', 'B4',
       'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35', 'C87', 'B77',
       'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19', 'B49', 'D',
       'C22 C26', 'C106', 'C65', 'E36', 'C54', 'B57 B59 B63 B66', 'C7',
       'E34', 'C32', 'B18', 'C124', 'C91', 'E40', 'T', 'C128', 'D37',
       'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44', 'A34', 'C104', 'C111',
       'C92', 'E38', 'D21', 'E12', 'E63', 'A14', 'B37', 'C30', 'D20',
       'B79', 'E25', 'D46', 'B73', 'C95', 'B38', 'B39', 'B22', 'C86',
       'C70', 'A16', 'C101', 'C68', 'A10', 'E68', 'B41', 'A20', 'D19',
       'D50', 'D9', 'A23', 'B50', 'A26', 'D48', 'E58', 'C126', 'B71',
       'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63', 'C62 C64', 'E24',

In [323]:
cabin = list(titanic.Cabin.unique()) #Obtain list of unqiue Cabin numbers

def uniqueLetters(cabin):
    '''Input list of unique Cabin numbers and output unique letters from entire list'''
    uniq_letters = set()
    for e in cabin:
        if type(e) is not str:
            next
        else:
            f = list(set(e))
            f = list(filter(lambda i: not str.isdigit(i), f))
            uniq_letters.update(f)
    uniq_letters.discard(" ") 
    return uniq_letters

uniqueLetters(cabin)

{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}

### Create Columns for Unique Cabin Letters 

In [324]:
uniqueLetters = uniqueLetters(cabin) #Retrieve unique 

#Create numeric binary columns for each letter
for letter in uniqueLetters:
    titanic["Cabin_" + letter] = 0

#Fill data for each column letter
for letter in uniqueLetters: #Iterate over each unqiue letter from Cabin number in data
    for index, row in titanic.iterrows(): #Iterate over each row of data
        if type(row["Cabin"]) != float: #Continue if Cabin value is not floating, which would mean not NaN
            if row["Cabin"].find(letter) != -1: #Look to see if unique letter is in Cabin value for row
                titanic.loc[index,"Cabin_" + letter] = 1 #Assign 1 if unique Cabin letter is found
                
titanic = titanic.drop('Cabin', 1) #drop Cabin column

In [325]:
titanic.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,...,Ticket_Prefix,Cabin_Yes,Cabin_E,Cabin_C,Cabin_F,Cabin_A,Cabin_T,Cabin_B,Cabin_G,Cabin_D
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,...,A5,0,0,0,0,0,0,0,0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,...,PC,1,0,1,0,0,0,0,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,...,STONO2,0,0,0,0,0,0,0,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,...,,1,0,1,0,0,0,0,0,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,...,,0,0,0,0,0,0,0,0,0


### Drop Unused Columns

In [326]:
titanic = titanic.drop(["PassengerId","Name"], 1)

In [327]:
titanic.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Ticket_Num,Ticket_Prefix,Cabin_Yes,Cabin_E,Cabin_C,Cabin_F,Cabin_A,Cabin_T,Cabin_B,Cabin_G,Cabin_D
0,0,3,male,22.0,1,0,7.2500,S,21171.0,A5,0,0,0,0,0,0,0,0,0
1,1,1,female,38.0,1,0,71.2833,C,17599.0,PC,1,0,1,0,0,0,0,0,0
2,1,3,female,26.0,0,0,7.9250,S,3101282.0,STONO2,0,0,0,0,0,0,0,0,0
3,1,1,female,35.0,1,0,53.1000,S,113803.0,,1,0,1,0,0,0,0,0,0
4,0,3,male,35.0,0,0,8.0500,S,373450.0,,0,0,0,0,0,0,0,0,0


### Add Dummy Variables for Sex, Embarked, and Ticket_Prefix

In [328]:
#Add dummy variables
titanic = titanic.join(pd.get_dummies(titanic["Sex"]))
titanic = titanic.join(pd.get_dummies(titanic["Embarked"], prefix="Embarked"))
titanic = titanic.join(pd.get_dummies(titanic["Ticket_Prefix"], prefix="Ticket_Prefix"))
titanic = titanic.drop(["Sex","Embarked","Ticket_Prefix"], 1)
titanic.head(5)

,Survived,Pclass,Age,SibSp,Parch,Fare,Ticket_Num,Cabin_Yes,Cabin_E,Cabin_C,...,Ticket_Prefix_SOP,Ticket_Prefix_SOPP,Ticket_Prefix_SOTONO2,Ticket_Prefix_SOTONOQ,Ticket_Prefix_SP,Ticket_Prefix_STONO 2,Ticket_Prefix_STONO2,Ticket_Prefix_SWPP,Ticket_Prefix_WC,Ticket_Prefix_WEP
0,0,3,22.0,1,0,7.2500,21171.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,38.0,1,0,71.2833,17599.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3,26.0,0,0,7.9250,3101282.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,1,35.0,1,0,53.1000,113803.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,3,35.0,0,0,8.0500,373450.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [329]:
titanic.describe()

C:\Program Files\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Survived,Pclass,Age,SibSp,Parch,Fare,Ticket_Num,Cabin_Yes,Cabin_E,Cabin_C,...,Ticket_Prefix_SOP,Ticket_Prefix_SOPP,Ticket_Prefix_SOTONO2,Ticket_Prefix_SOTONOQ,Ticket_Prefix_SP,Ticket_Prefix_STONO 2,Ticket_Prefix_STONO2,Ticket_Prefix_SWPP,Ticket_Prefix_WC,Ticket_Prefix_WEP
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,8.870000e+02,891.000000,891.000000,891.000000,...,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,2.983284e+05,0.228956,0.037037,0.066218,...,0.001122,0.003367,0.002245,0.016835,0.001122,0.013468,0.006734,0.002245,0.011223,0.003367
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429,6.576144e+05,0.420397,0.188959,0.248802,...,0.033501,0.057961,0.047351,0.128725,0.033501,0.115332,0.081830,0.047351,0.105403,0.057961
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,3.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,NaN,0.000000,0.000000,7.910400,NaN,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,3.000000,NaN,0.000000,0.000000,14.454200,NaN,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,3.000000,NaN,1.000000,0.000000,31.000000,NaN,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,3.101317e+06,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [331]:
titanic.isnull().sum()

Survived                      0
Pclass                        0
Age                         177
SibSp                         0
Parch                         0
Fare                          0
Ticket_Num                    4
Cabin_Yes                     0
Cabin_E                       0
Cabin_C                       0
Cabin_F                       0
Cabin_A                       0
Cabin_T                       0
Cabin_B                       0
Cabin_G                       0
Cabin_D                       0
female                        0
male                          0
Embarked_C                    0
Embarked_Q                    0
Embarked_S                    0
Ticket_Prefix_                0
Ticket_Prefix_A4              0
Ticket_Prefix_A5              0
Ticket_Prefix_AS              0
Ticket_Prefix_C               0
Ticket_Prefix_CA              0
Ticket_Prefix_CASOTON         0
Ticket_Prefix_FC              0
Ticket_Prefix_FCC             0
Ticket_Prefix_Fa              0
Ticket_P

### Write Data

In [330]:
filepath = "Data/train_wrangled.csv"
titanic.to_csv(filepath)